In [ ]:
from crewai import Agent, Task, Crew

## Agentes

In [ ]:
representante_vendas = Agent(
    role="Representante de Vendas",
    goal="Identificar leads de alto valor que correspondam "
         "ao nosso perfil de cliente ideal",
    backstory=(
        "Como parte da dinâmica equipe de vendas da CrewAI, "
        "sua missão é vasculhar "
        "o cenário digital em busca de leads potenciais. "
        "Armado com ferramentas de ponta "
        "e uma mentalidade estratégica, você analisa dados, "
        "tendências e interações para "
        "descobrir oportunidades que outros possam ter negligenciado. "
        "Seu trabalho é crucial para abrir caminho "
        "para engajamentos significativos e impulsionar o crescimento da empresa."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lider_vendas = Agent(
    role="Líder de Vendas",
    goal="Cultivar leads com comunicações personalizadas e cativantes",
    backstory=(
        "Dentro do vibrante ecossistema do departamento de vendas da CrewAI, "
        "você se destaca como a ponte entre os clientes potenciais "
        "e as soluções que eles precisam."
        "Criando mensagens envolventes e personalizadas, "
        "você não apenas informa os leads sobre nossas ofertas "
        "mas também os faz sentir vistos e ouvidos."
        "Seu papel é crucial para converter interesse "
        "em ação, guiando os leads na jornada "
        "da curiosidade ao compromisso."
    ),
    allow_delegation=False,
    verbose=True
)


## Ferramentas

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Criando Ferramenta Customizada

In [ ]:
from crewai_tools import BaseTool
import json

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Ferramenta de Análise de Sentimento"
    description: str = ("Analisa o sentimento do texto "
                        "para garantir uma comunicação positiva e envolvente.")
    
    def _run(self, text: str) -> str:
        # Seu código personalizado para a ferramenta vai aqui
        return "positivo"

In [ ]:
class SearchTool(BaseTool):
    name: str = "Ferramenta para busca de sites"
    description: str = ("Retorna um JSON com os sites das empresas.")
    
    def _run(self) -> str:
        return json.dumps({
            "results": [
                {
                    "description": "A AetherTech é uma empresa líder em tecnologia de energia espacial, desenvolvendo soluções sustentáveis e inovadoras para o futuro.  A empresa está construindo uma infraestrutura de energia solar orbital,  oferecendo  uma  fonte limpa e ininterrupta de energia  para  o  planeta.",
                    "url": "http://127.0.0.1:5000/aether-tech"
                },
                {
                    "description": "Solaris Ventures está  revolucionando  o  turismo  espacial  com  o  primeiro  hotel  espacial  de  luxo  do  mundo,  oferecendo  experiências  inesquecíveis  para  quem  busca  aventuras  interestelares  em  um  ambiente  único  e  luxuoso.",
                    "url": "http://127.0.0.1:5000/solaris-ventures"
                }
            ]
        })

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Criando Tasks

In [ ]:
representante_vendas_task = Task(
    description=(
        "Conduza uma análise aprofundada de {lead_name}, "
        "uma empresa no setor de {industry} "
        "que recentemente demonstrou interesse em nossas soluções. "
        "Utilize todas as fontes de dados disponíveis "
        "para compilar um perfil detalhado, "
        "focando em principais tomadores de decisão, desenvolvimentos recentes da empresa "
        "e necessidades potenciais "
        "que se alinhem com nossas ofertas. "
        "Esta tarefa é crucial para adaptar "
        "nossa estratégia de engajamento de forma eficaz.\n"
        "Não faça suposições e "
        "use apenas informações das quais você tem absoluta certeza."
    ),
    expected_output=(
        "Um relatório abrangente sobre {lead_name}, "
        "incluindo histórico da empresa, "
        "principais funcionários, marcos recentes e necessidades identificadas. "
        "Destaque as áreas potenciais onde "
        "nossas soluções podem fornecer valor "
        "e sugira estratégias de engajamento personalizadas."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=representante_vendas,
)


In [ ]:
lider_vendas_task = Task(
    description=(
        "Usando os insights obtidos do "
        "relatório de perfil de lead sobre {lead_name}, "
        "crie uma campanha de contato personalizada "
        "direcionada a {key_decision_maker}, "
        "o(a) {position} de {lead_name}. "
        "A campanha deve abordar o recente {milestone} deles "
        "e como nossas soluções podem apoiar seus objetivos. "
        "Sua comunicação deve ressoar "
        "com a cultura e os valores da empresa {lead_name}, "
        "demonstrando um profundo entendimento "
        "do negócio e das necessidades deles.\n"
        "Não faça suposições e use apenas "
        "informações das quais você tem absoluta certeza."
    ),
    expected_output=(
        "Uma série de rascunhos de e-mails personalizados "
        "adaptados para {lead_name}, "
        "especificamente direcionados a {key_decision_maker}."
        "Cada rascunho deve incluir "
        "uma narrativa envolvente que conecte nossas soluções "
        "com as conquistas recentes e os objetivos futuros deles. "
        "Garanta que o tom seja envolvente, profissional "
        "e alinhado com a identidade corporativa de {lead_name}."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lider_vendas,
)


## Creew

In [ ]:
crew = Crew(
    agents=[representante_vendas, 
            lider_vendas],
    
    tasks=[representante_vendas_task, 
           lider_vendas_task],
	
    verbose=2,
	memory=True
)

## Executando a Creew

inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

### Resultado

In [ ]:
from IPython.display import Markdown
Markdown(result)